In [ ]:
%pip install datasets

In [ ]:
import jax
import math
from tqdm import tqdm
from torch import load
from requests import get
from jax.lax import pmean
from flax import linen as nn
from functools import partial
from flax import traverse_util
import matplotlib.pyplot as plt
from datasets import load_dataset
from dataclasses import dataclass
from flax.jax_utils import replicate
from transformers import AutoTokenizer
from jax import Array, pmap, jit, random
from jax.nn.initializers import lecun_normal
from flax.training.train_state import TrainState
from jax import numpy as jnp, ensure_compile_time_eval
from flax.training.orbax_utils import save_args_from_target
from orbax.checkpoint import Checkpointer, PyTreeCheckpointHandler
from jax import numpy as jnp, value_and_grad, ensure_compile_time_eval
from optax import adamw, set_to_zero, multi_transform, linear_schedule, join_schedules, constant_schedule

jax.devices()

In [ ]:
@dataclass
class PhiConfig:
    n_head: int = 32
    n_layer: int = 24
    n_embed: int = 2048
    rotary_dim: int = 32
    ln_eps: float = 1e-5
    n_positions: int = 2048
    vocab_size: int = 51200
    warmup_steps: int = 500
    target_hidden_size: int = 2048
    param_dtype: jnp.dtype = jnp.bfloat16

In [ ]:
def compute_cos_sin(config: PhiConfig) -> (Array, Array):
    t = jnp.arange(config.n_positions, dtype=jnp.float32)
    inv_freq = 1 / (10000 ** (jnp.arange(0, config.rotary_dim, 2, dtype=jnp.float32) / config.rotary_dim))
    freqs = jnp.outer(t, inv_freq)
    return jnp.cos(freqs).astype(config.param_dtype), jnp.sin(freqs).astype(config.param_dtype)

def apply_rotary_emb(qkv: Array, cos: Array, sin: Array) -> Array:
    _, seq_len, _, _, _ = qkv.shape
    _, rotary_dim = cos.shape
    rotary_dim *= 2

    q_rot = qkv[:, :, 0, :, :rotary_dim]
    q_pass = qkv[:, :, 0, :, rotary_dim:]

    k_rot = qkv[:, :, 1, :, :rotary_dim]
    k_pass = qkv[:, :, 1, :, rotary_dim:]

    q1, q2 = jnp.split(q_rot.astype(jnp.float32), 2, axis=-1)
    k1, k2 = jnp.split(k_rot.astype(jnp.float32), 2, axis=-1)
    c, s = cos[:seq_len][:, None, :].astype(jnp.float32), sin[:seq_len][:, None, :].astype(jnp.float32)

    q_rot = jnp.concatenate([q1 * c - q2 * s, q1 * s + q2 * c], axis=-1).astype(qkv.dtype)
    k_rot = jnp.concatenate([k1 * c - k2 * s, k1 * s + k2 * c], axis=-1).astype(qkv.dtype)

    return jnp.concatenate([
        jnp.concatenate([q_rot, q_pass], axis=-1)[:, :, None, :, :],
        jnp.concatenate([k_rot, k_pass], axis=-1)[:, :, None, :, :],
        qkv[:, :, 2:3, :, :]
    ], axis=2)

class SelfAttention(nn.Module):
    config: PhiConfig

    @nn.compact
    def __call__(self, x: Array) -> Array:
        batch_size, seq_len, n_embed = x.shape

        with ensure_compile_time_eval():
            cos, sin = compute_cos_sin(self.config)

        qkv = nn.Dense(features=3 * self.config.n_embed, use_bias=True)(x)
        qkv = jnp.reshape(qkv, (batch_size, seq_len, 3, self.config.n_head, n_embed // self.config.n_head))
        qkv = apply_rotary_emb(qkv, cos, sin)
        q, k, v = jnp.split(qkv, 3, axis=2)
        q, k, v = jnp.squeeze(q, axis=2), jnp.squeeze(k, axis=2), jnp.squeeze(v, axis=2)
        q, k = q.astype(jnp.float32), k.astype(jnp.float32)

        scale = 1.0 / math.sqrt(q.shape[-1])
        s = jnp.einsum("bthd,bshd->bhts", q, k * scale)
        s = s + jnp.triu(jnp.full((seq_len, seq_len), -10000.0), 1)
        a = nn.softmax(s, axis=-1).astype(v.dtype)
        a = jnp.einsum("bhts,bshd->bthd", a, v)
        a = jnp.reshape(a, (batch_size, seq_len, n_embed))
        return nn.Dense(features=n_embed, use_bias=True)(a)

class MLP(nn.Module):
    config: PhiConfig

    @nn.compact
    def __call__(self, x: Array, l: Array) -> (Array, Array, Array):
        h = nn.Dense(features=self.config.n_embed * 4, use_bias=True)(x)
        h = nn.Dense(features=self.config.n_embed, use_bias=True)(nn.gelu(h, approximate=True))

        d1 = nn.Dense(features=self.config.target_hidden_size, use_bias=True)
        d2 = nn.Dense(features=self.config.n_embed, use_bias=True)
        
        lp = d2(nn.gelu(d1(x), approximate=True))
        lg = d2(nn.gelu(d1(l), approximate=True))
        return (h, lg, jnp.mean((lp - h)**2))

class Block(nn.Module):
    config: PhiConfig

    @nn.compact
    def __call__(self, x: Array, l: Array) -> (Array, Array, Array):
        ln = nn.LayerNorm(epsilon=self.config.ln_eps)
        sa = SelfAttention(self.config)
        ah = sa(ln(x))
        al = sa(ln(l))
        h, l, loss = MLP(self.config)(ah, al)
        return (ah + h + x, al + l + x, loss)

class Phi(nn.Module):
    config: PhiConfig

    @nn.compact
    def __call__(self, x: Array) -> (Array, Array, Array):
        losses = 0
        h = nn.Embed(num_embeddings=self.config.vocab_size, features=self.config.n_embed)(x)
        l = h
        for _ in range(self.config.n_layer):
            (h, l, loss) = Block(self.config)(h, l)
            losses += loss
        # useless layers while training
        ln = nn.LayerNorm(epsilon=self.config.ln_eps)
        d = nn.Dense(self.config.vocab_size, use_bias=True)
        h = d(ln(h)).astype(jnp.float32)
        l = d(ln(l)).astype(jnp.float32)
        return (h, l, losses)

In [ ]:
def load_model_into_flax(config, model_path) -> dict:
    print("loading pytorch model")
    with open("/kaggle/working/pytorch_model.bin", "wb") as f:
        f.write(get(model_path, allow_redirects=True).content)
    model = load("/kaggle/working/pytorch_model.bin", map_location="cpu")
    print("pytorch model loaded")
    initializer = lecun_normal()
    print("loading model into flax")
    print("init trainable params")
    params = {}
    params["Embed_0"] = {}
    params["LayerNorm_0"] = {}
    params["Dense_0"] = {}
    for i in range(config.n_layer):
        params[f"Block_{i}"] = {}
        params[f"Block_{i}"]["MLP_0"] = {}
        params[f"Block_{i}"]["MLP_0"]["Dense_0"] = {}
        params[f"Block_{i}"]["MLP_0"]["Dense_1"] = {}
        params[f"Block_{i}"]["MLP_0"]["Dense_2"] = {}
        params[f"Block_{i}"]["MLP_0"]["Dense_3"] = {}
        params[f"Block_{i}"]["LayerNorm_0"] = {}
        params[f"Block_{i}"]["SelfAttention_0"] = {}
        params[f"Block_{i}"]["SelfAttention_0"]["Dense_0"] = {}
        params[f"Block_{i}"]["SelfAttention_0"]["Dense_1"] = {}

    for i in range(config.n_layer):
        params[f"Block_{i}"]["MLP_0"]["Dense_2"]["kernel"] = initializer(random.PRNGKey(0), (config.n_embed, config.target_hidden_size), dtype=config.param_dtype)
        params[f"Block_{i}"]["MLP_0"]["Dense_2"]["bias"] = jnp.zeros((config.target_hidden_size,), dtype=config.param_dtype)
        params[f"Block_{i}"]["MLP_0"]["Dense_3"]["kernel"] = initializer(random.PRNGKey(0), (config.target_hidden_size, config.n_embed), dtype=config.param_dtype)
        params[f"Block_{i}"]["MLP_0"]["Dense_3"]["bias"] = jnp.zeros((config.n_embed,), dtype=config.param_dtype)
    print("init frozen params")
    for param_name in model:
        jnp_array = jnp.asarray(model[param_name]).astype(config.param_dtype)
        if "transformer" in param_name:
            if "embd" in param_name:
                params["Embed_0"]["embedding"] = jnp_array
            else:
                param_name = param_name.replace("transformer.h", "")
                layer = int(param_name.split(".")[1])
                param_name = param_name.replace(f".{layer}.", "")
                match param_name:
                    case "ln.weight": params[f"Block_{layer}"]["LayerNorm_0"]["scale"] = jnp_array
                    case "ln.bias": params[f"Block_{layer}"]["LayerNorm_0"]["bias"] = jnp_array
                    case "mixer.Wqkv.weight": params[f"Block_{layer}"]["SelfAttention_0"]["Dense_0"]["kernel"] = jnp_array.T
                    case "mixer.Wqkv.bias": params[f"Block_{layer}"]["SelfAttention_0"]["Dense_0"]["bias"] = jnp_array
                    case "mixer.out_proj.weight": params[f"Block_{layer}"]["SelfAttention_0"]["Dense_1"]["kernel"] = jnp_array.T
                    case "mixer.out_proj.bias": params[f"Block_{layer}"]["SelfAttention_0"]["Dense_1"]["bias"] = jnp_array
                    case "mlp.fc1.weight": params[f"Block_{layer}"]["MLP_0"]["Dense_0"]["kernel"] = jnp_array.T
                    case "mlp.fc1.bias": params[f"Block_{layer}"]["MLP_0"]["Dense_0"]["bias"] = jnp_array
                    case "mlp.fc2.weight": params[f"Block_{layer}"]["MLP_0"]["Dense_1"]["kernel"] = jnp_array.T
                    case "mlp.fc2.bias": params[f"Block_{layer}"]["MLP_0"]["Dense_1"]["bias"] = jnp_array
        else:
            param_name = param_name.replace("lm_head.", "")
            match param_name:
                case "ln.weight": params["LayerNorm_0"]["scale"] = jnp_array
                case "ln.bias": params["LayerNorm_0"]["bias"] = jnp_array
                case "linear.weight": params["Dense_0"]["kernel"] = jnp_array.T
                case "linear.bias": params["Dense_0"]["bias"] = jnp_array
    print("model loaded into flax")
    return params

def init_train_state(config, model_path) -> TrainState:
    phi = Phi(config)
    params = load_model_into_flax(config, model_path)

    lr = linear_schedule(init_value=0, end_value=1e-3, transition_steps=config.warmup_steps)
    partition_optimizers = {"trainable": adamw(lr), "frozen": set_to_zero()}
    param_partitions = traverse_util.path_aware_map(lambda path, _: "trainable" if ("Dense_2" in path or "Dense_3" in path) else "frozen", params)

    state = TrainState.create(
        apply_fn=phi.apply,
        tx=multi_transform(partition_optimizers, param_partitions),
        params=params
    )
    return replicate(state)

In [ ]:
@partial(pmap, axis_name="device")
def train_step(state: TrainState, batch: Array):
    def loss_fn(params):
        h, l, loss = state.apply_fn({"params": params}, batch)
        t = nn.one_hot(jnp.argmax(h, axis=-1), h.shape[-1])
        return loss + jnp.mean(-jnp.sum(t * nn.log_softmax(l), axis=-1)) # jnp.mean((h - l)**2)
    
    grad_fn = value_and_grad(loss_fn)
    loss, grads = grad_fn(state.params)
    grads = pmean(grads, axis_name="device")
    loss = pmean(loss, axis_name="device")
    return (state.apply_gradients(grads=grads), loss)

In [ ]:
def train_epoch(state: TrainState, n_steps: int, batch_size: int, config: PhiConfig) -> TrainState:
    dataset = load_dataset("HuggingFaceH4/ultrachat_200k", split="train_gen")
    tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5")
    tokenizer.pad_token = tokenizer.eos_token
    n_devices = jax.local_device_count()

    loss = 0
    losses = []
    batch_size = batch_size * n_devices
    tqdm_range = tqdm(range(n_steps))
    for i in tqdm_range:
        batch = jnp.array(tokenizer([dataset[i * batch_size + j]["prompt"] for j in range(batch_size)], padding="max_length", max_length=config.n_positions, truncation=True)["input_ids"], dtype=jnp.int32)
        batch = jax.tree_map(lambda x: x.reshape((n_devices, -1, *x.shape[1:])), batch)

        state, loss = train_step(state, batch)
        tqdm_range.set_description(f"Loss: {loss.mean():.4f}")
        losses.append(loss.mean())

    plt.plot(losses)
    return state

In [ ]:
batch_size = 2
config = PhiConfig()
state = init_train_state(config, "https://huggingface.co/microsoft/phi-1_5/resolve/main/pytorch_model.bin")
state = train_epoch(state, 8000, batch_size, config)

In [ ]:
import torch
import numpy as np

print("saving flax model")
model = torch.load("/kaggle/working/pytorch_model.bin", map_location="cpu")

for param_name in tqdm(model):
    if "transformer" in param_name:
        if not "embd" in param_name:
            o_param_name = param_name
            param_name = param_name.replace("transformer.h", "")
            layer = int(param_name.split(".")[1])
            param_name = param_name.replace(f".{layer}.", "")
            match param_name:
                case "mlp.fc1.weight": model[o_param_name] = torch.from_numpy(np.array(state.params[f"Block_{layer}"]["MLP_0"]["Dense_2"]["kernel"].T.astype(jnp.float16), dtype=np.float16))
                case "mlp.fc1.bias": model[o_param_name] = torch.from_numpy(np.array(state.params[f"Block_{layer}"]["MLP_0"]["Dense_2"]["bias"].astype(jnp.float16), dtype=np.float16))
                case "mlp.fc2.weight": model[o_param_name] = torch.from_numpy(np.array(state.params[f"Block_{layer}"]["MLP_0"]["Dense_3"]["kernel"].T.astype(jnp.float16), dtype=np.float16))
                case "mlp.fc2.bias": model[o_param_name] = torch.from_numpy(np.array(state.params[f"Block_{layer}"]["MLP_0"]["Dense_3"]["bias"].astype(jnp.float16), dtype=np.float16))

torch.save(model, "/kaggle/working/flax_model.bin")